In [23]:
import urllib # dataya erişim
import json # data formatının dönüşümü
import pandas as pd 
import numpy as np
from scipy.stats import t


In [2]:
url = 'https://data.ibb.gov.tr/datastore/odata3.0/25077460-ddfb-45b6-b32c-4615f0ad2d57?$format=json'  #URL of family size according to districts

def ibbVeriToDataframe(url):
    # API connection
    sorgu = urllib.request.urlopen(url)
    # Change to json type
    data = json.loads(sorgu.read().decode())
    # Only getting data value
    data = data.get("value")
    # Verinin dataframe formatına dönüştürülmesi
    return pd.DataFrame(data)

In [3]:

df = ibbVeriToDataframe(url)

print(df.columns) 
df = df.drop(columns="_id") # Removal of id column

Index(['_id', 'Ilce Adi', 'Ortalama Hanehalki Buyuklugu'], dtype='object')


In [4]:
df.head() #Dataframe  family size according to districts

,Ilce Adi,Ortalama Hanehalki Buyuklugu
0,Adalar,2.54
1,Bakırköy,2.87
2,Beşiktaş,2.47
3,Beykoz,3.29
4,Beyoğlu,3.11


In [5]:
kadikoy_list = ['Kadıköy', 'Kartal', 'Şile', 'Üsküdar', 'Pendik', 'Ümraniye', 'Maltepe', 'Sultanbeyli', 'Tuzla', 'Ataşehir', 'Çekmeköy', 'Sancaktepe', 'Beykoz']
besiktas_list = ['Beşiktaş', 'Bakırköy', 'Beyoğlu', 'Çatalca', 'Eyüp', 'Fatih', 'Gaziosmanpaşa', 'Sarıyer', 'Silivri', 'Şişli', 'Zeytinburnu', 'Büyükçekmece', 'Kağıthane', 'Küçükçekmece', 'Bayrampaşa', 'Avcılar', 'Bağcılar', 'Arnuvutköy', 'Başakşehir', 'Beylikdüzü', 'Esenyurt', 'Sultangazi', 'Esenler', 'Güngören', 'Bahçelievler']

kadikoy_dist = []
besiktas_dist = []

In [6]:
import requests, json
  

api_key ='your_api_key'
    
# url variable store url 
url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'

for dest in kadikoy_list:
    source = 'kadiköy'
    r = requests.get(url + 'origins=' + source +
                   '&destinations=' + dest +
                   '&key=' + api_key)
    kadikoy_dist.append(r.json()['rows'][0]['elements'][0]['distance']['value'])

for dest in besiktas_list:
    source = 'besiktas'
    r = requests.get(url + 'origins=' + source +
                   '&destinations=' + dest +
                   '&key=' + api_key)
    besiktas_dist.append(r.json()['rows'][0]['elements'][0]['distance']['value'])

In [7]:
data_asian = pd.DataFrame({'Districts': kadikoy_list, 'Distances': kadikoy_dist})
data_european = pd.DataFrame({'Districts': besiktas_list, 'Distances': besiktas_dist})

In [8]:
data_european.head()

,Districts,Distances
0,Beşiktaş,0
1,Bakırköy,20577
2,Beyoğlu,3823
3,Çatalca,65300
4,Eyüp,24906


In [9]:
data_european['Distances'][0] = 1500    #Not logical to keep it 0, so assigned 1500.
data_asian['Distances'][0] = 1500   #Not logical to keep it 0, so assigned 1500.

/Users/emreuzel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/emreuzel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
distance_data = pd.concat([data_european, data_asian], ignore_index = True) #Concat european and asian side districts in one dataframe

In [11]:
df = df.drop(0) 
df.rename(columns={"Ilce Adi": "Districts", "Ortalama Hanehalki Buyuklugu": "Average Family Size"},inplace =True) #Change to english columns
df.reset_index().drop(columns = ['index'])

,Districts,Average Family Size
0,Bakırköy,2.87
1,Beşiktaş,2.47
2,Beykoz,3.29
3,Beyoğlu,3.11
4,Çatalca,2.79
5,Eyüp,3.32
6,Fatih,3.21
7,Gaziosmanpaşa,3.61
8,Kadıköy,2.49
9,Kartal,3.20


In [12]:
last_data = pd.merge(df, distance_data, on = 'Districts')

0          Bakırköy
1          Beşiktaş
2            Beykoz
3           Beyoğlu
4           Çatalca
5              Eyüp
6             Fatih
7     Gaziosmanpaşa
8           Kadıköy
9            Kartal
10          Sarıyer
11          Silivri
12             Şile
13            Şişli
14          Üsküdar
15      Zeytinburnu
16     Büyükçekmece
17        Kağıthane
18     Küçükçekmece
19           Pendik
20         Ümraniye
21       Bayrampaşa
22          Avcılar
23         Bağcılar
24     Bahçelievler
25         Güngören
26          Maltepe
27      Sultanbeyli
28            Tuzla
29          Esenler
30         Ataşehir
31       Başakşehir
32       Beylikdüzü
33         Çekmeköy
34         Esenyurt
35       Sancaktepe
36       Sultangazi
Name: Districts, dtype: object

In [13]:
last_data['Distances'] = last_data['Distances'] / 10 # Don't want a high difference between distances

In [14]:
last_data.head()

,Districts,Average Family Size,Distances
0,Bakırköy,2.87,2057.7
1,Beşiktaş,2.47,150.0
2,Beykoz,3.29,2836.2
3,Beyoğlu,3.11,382.3
4,Çatalca,2.79,6530.0


## Control Variables

- Amount of Waste by District, Year and Type of Waste

- Amount of Gas Consumption by District

In [22]:
# Waste by District
url = 'https://data.ibb.gov.tr/datastore/odata3.0/50036dfd-aea5-4f06-832f-f7020fdaaa5a?$format=json'
waste_df = ibbVeriToDataframe(url)

print(waste_df.columns) 
waste_df = waste_df.drop(columns="_id") # ID removal

Index(['_id', 'Ilce', 'Veri Turu', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020'],
      dtype='object')


In [23]:
waste_df = waste_df.drop(0)[['Ilce', '2020']] #Drop Adalar as it is specified as outlier and other years

In [24]:
waste_df.rename(columns = {'Ilce': 'Districts', '2020': 'Waste'}, inplace = True)
waste_df.head()

,Districts,Waste
1,Arnavutköy,31498125.0
2,Ataşehir,21541611.0
3,Avcılar,22698126.0
4,Bahçelievler,11198229.0
5,Bağcılar,9418054.5


In [25]:
last_data = pd.merge(last_data, waste_df, on = 'Districts')
last_data.head()

,Districts,Average Family Size,Distances,Waste
0,Bakırköy,2.87,2057.7,39789365.0
1,Beşiktaş,2.47,150.0,36745978.5
2,Beykoz,3.29,2836.2,23152626.0
3,Beyoğlu,3.11,382.3,50934234.0
4,Çatalca,2.79,6530.0,7699068.0


### Gas Consumption

In [26]:
# Gas Consumption by District
url = 'https://data.ibb.gov.tr/datastore/odata3.0/d5fe41b0-3848-4548-9ac7-6e4756c3027b?$format=json'
gas_df = ibbVeriToDataframe(url)

print(gas_df.columns) 
gas_df = gas_df.drop(columns="_id") # ID removal, its extra

Index(['_id', 'Ilce', '2015.0', '2016.0', '2017.0', '2018.0', '2019.0'], dtype='object')


In [27]:
gas_df = gas_df.drop(0)[['Ilce', '2019.0']]

In [28]:
gas_df.head() #Upper case need to change it to tile case

,Ilce,2019.0
1,ESENYURT,270391774
2,PENDİK,260462353
3,KÜÇÜKÇEKMECE,256603576
4,ÜMRANİYE,237214965
5,ÜSKÜDAR,210159134


In [29]:
gas_df.rename(columns = {'Ilce': 'Districts', '2019.0': 'GasConsumption'}, inplace = True)
gas_df.head()

,Districts,GasConsumption
1,ESENYURT,270391774
2,PENDİK,260462353
3,KÜÇÜKÇEKMECE,256603576
4,ÜMRANİYE,237214965
5,ÜSKÜDAR,210159134
6,BAHÇELİEVLER,200595190
7,BAĞCILAR,194966321
8,SARIYER,185898510
9,BAKIRKÖY,180569905
10,MALTEPE,171969204


In [30]:
gas_df['Districts'] = gas_df['Districts'].str.title() #Needs to be title case for join

In [31]:
gas_df['Districts'] #There are some string case problems to turkish characters. That's why corrected them below
gas_df['Districts'][2] = 'Pendik'
gas_df['Districts'][4] = 'Ümraniye'
gas_df['Districts'][6] = 'Bahçelievler'
gas_df['Districts'][7] = 'Bağcılar'
gas_df['Districts'][8] = 'Sarıyer'
gas_df['Districts'][9] = 'Bakırköy'
gas_df['Districts'][11] = 'Şişli'
gas_df['Districts'][12] = 'Fatih'
gas_df['Districts'][13] = 'Beşiktaş'
gas_df['Districts'][14] = 'Ataşehir'
gas_df['Districts'][17] = 'Kağıthane'
gas_df['Districts'][18] = 'Avcılar'
gas_df['Districts'][20] = 'Gaziosmanpaşa'
gas_df['Districts'][21] = 'Beylikdüzü'
gas_df['Districts'][22] = 'Sultangazi'
gas_df['Districts'][23] = 'Başakşehir'
gas_df['Districts'][29] = 'Zeytinburnu'
gas_df['Districts'][33] = 'Silivri'
gas_df['Districts'][34] = 'Sultanbeyli'
gas_df['Districts'][36] = 'Şile'


/Users/emreuzel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/emreuzel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/emreuzel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [33]:
last_data = pd.merge(last_data, gas_df, on = 'Districts')
last_data.head() #Final Data, so I will write this csv to as final data and will use it in my function.

,Districts,Average Family Size,Distances,Waste,GasConsumption
0,Bakırköy,2.87,2057.7,39789365.0,180569905
1,Beşiktaş,2.47,150.0,36745978.5,155364950
2,Beykoz,3.29,2836.2,23152626.0,134096902
3,Beyoğlu,3.11,382.3,50934234.0,102610446
4,Çatalca,2.79,6530.0,7699068.0,5845883


In [34]:
# last_data.to_csv('last_data_ready', index = False)

# FUNCTION

In [58]:
from linear_analysis import the_function #import the function for linear analysis

In [54]:
all_B, standard_error_B, t_confidence_intervals, VarOfInterest = the_function('Average Family Size', 'Districts', 'Distances')

# Hypothesis 

We are curious about  whether the distances of districts have significant effect on predicting Average Family Size

- H0 Hypothesis: Distances doesn't have effect determining Average Family Size
- H1 Hypothesis: Distances have effect determining Average Family Size.

To achieve this the abs(t0) value must be out of confidence intervals of t distribution for specified alpha = 0.05 and n-p-1. 

Computationaly:
    
    -IF abs(t0) > confidence_interval_value1 and  abs(t0) > confidence_interval_value2; then it has high effect reject null hypothesis
    
    -Otherwise; Reject Null Hypothesis

We can change District's column name and calculate for other variables as well.



In [62]:
t0 = all_B[VarOfInterest] / standard_error_B[VarOfInterest] #Also calculating t0 value to prove hypothesis
t0

-0.8044282709813296

In [71]:
confidence_interval_value1, confidence_interval_value2 =  t_confidence_intervals

In [66]:
if np.abs(t0) > np.abs(confidence_interval_value1): #Does not need to look for both interval sides
    print('Reject null hypothesis! There is significant effect.')
else:
    print('There are not enough evidence to reject null hypthosis')
    

There are not enough evidence to reject null hypthosis


# Regression Table

In [72]:
big_credible_intervals = [] #We are %95 confident that variables are between these intervals
for i in range(len(standard_error_B)):
    little_credible_intervals = []
    little_credible_intervals.append(all_B[i] - np.abs(confidence_interval_value1)*standard_error_B[i])
    little_credible_intervals.append(all_B[i] + np.abs(confidence_interval_value1)*standard_error_B[i])
    big_credible_intervals.append(little_credible_intervals)

In [89]:
lower_= [] #Lower limit of intervals
upper_= [] #Upper limit of intervals
for i in big_credible_intervals:
    lower_.append(i[0])
    upper_.append(i[1])

In [122]:
reg_table = pd.DataFrame()
reg_table['B coefficents'] = all_B
reg_table['Standard Errors of Coefficents'] = standard_error_B
reg_table['Lower Bound of Credible Interval B'] = lower_
reg_table['Upper Bound of Credible Interval B'] = upper_
reg_table.index = (['B0', 'B1', 'B2', 'B3'])

In [123]:
reg_table

,B coefficents,Standard Errors of Coefficents,Lower Bound of Credible Interval B,Upper Bound of Credible Interval B
B0,3.467671e+00,2.748991e-01,2.907011e+00,4.028331e+00
B1,-3.451880e-05,4.291097e-05,-1.220363e-04,5.299870e-05
B2,-1.030685e-08,4.907569e-09,-2.031591e-08,-2.978017e-10
B3,1.839594e-09,1.318947e-09,-8.504161e-10,4.529604e-09
